In [1]:
!pip3 install tweepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
def age2tag(text):
  age = int(text[:2])
  if age < 20:
    tag = '<10代>'
  elif age < 40:
    tag = '<20～30代>'
  elif age < 70:
    tag = '<40～60代>'
  else:
    tag = '<None>'
  return tag

In [4]:
import tweepy
import random
import re


while True:
    api_key = 'yDw48aKG3Trf56AHoCNBp558l'
    api_secret_key = '2XU9UfJgUGlhKJwDe7dIIHOJZ828EZqheWs8niCV6jvH5TxiZg'
    access_token = '3041224710-AMaUeWnNloLRGcitnx5lAsIrdIh59jC5ehWkfeX'
    access_token_secret = '4238EOYuICrESgWiH3ZDx35o8hiE1QWPFPx061hygy14u'

    auth = tweepy.OAuthHandler(api_key, api_secret_key)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth_handler=auth, wait_on_rate_limit=True)

    # botのツイートを除外するため，一般的なクライアント名を列挙
    sources = ["TweetDeck", "Twitter Web Client", "Twitter for iPhone",
               "Twitter for iPad", "Twitter for Android", "Twitter for Android Tablets",
               "ついっぷる", "Janetter", "twicca", "Keitai Web", "Twitter for Mac"]
    

    # ひらがな一文字で検索し，スクリーンネームを取得
    words = list("あいうえおかきくけこさしすせそたちつてとなにぬねのはひふへほまみむめもやゆよらりるれろわをん")
    
    # screen_name（ユーザー名）を取得している
    screen_names = set()
    for s in api.search(q=random.choice(words), lang='ja', result_type='recent', count=100, tweet_mode='extended'):
        if s.source in sources:
            screen_names.add(s.author.screen_name)

    # ステータスidからステータスを得るためのdict
    id2status = {}

    # スクリーンネームからタイムラインを取得してツイートを保存．
    # さらにリプライツイートであれば，リプライ先のスクリーンネームも取得
    in_reply_to_screen_names = set()
    for name in screen_names:
        try:
            for s in api.user_timeline(name, tweet_mode='extended', count=200):
                # リンクもしくはハッシュタグを含むツイートは除外する
                if "http" not in s.full_text and "#" not in s.full_text:
                    id2status[s.id] = s
                    if s.in_reply_to_screen_name is not None:
                        if s.in_reply_to_screen_name not in screen_names:
                            in_reply_to_screen_names.add(s.in_reply_to_screen_name)
        except Exception as e:
            continue

    # リプライ先のスクリーンネームからタイムラインを取得してツイートを保存
    # 例）年齢を含むツイートを表示する
    pattern = re.compile('[1-6][0-9][\u6B73\u4EE3]')
    for name in in_reply_to_screen_names:
        try:
          user = api.get_user(screen_name=name)
          if re.search(pattern, user.description):
            for s in api.user_timeline(name, tweet_mode='extended', count=200):
                if "http" not in s.full_text and "#" not in s.full_text:
                    id2status[s.id] = s
        except Exception as e:
            continue

    # 保存したツイートのリプライ先のツイートが保存されていれば，id2replyidのキーを元ツイートのid，値をリプライ先ツイートのidとする
    id2replyid = {}
    for _, s in id2status.items():
        if s.in_reply_to_status_id in id2status:
            id2replyid[s.in_reply_to_status_id] = s.id

    # id2replyidのkey valueからstatusを取得し，ツイートペアをタブ区切りで保存
    # 例）年齢を含むツイートを表示する
    pattern = re.compile('[1-6][0-9][\u6B73\u4EE3]')
    f = open("tweet_pairs.txt", "a")
    for id, rid in id2replyid.items():
        user = api.get_user(screen_name=id2status[rid].author.screen_name)
        if re.search(pattern, user.description):
            # 改行は半角スペースに置換
            tweet1 = id2status[id].full_text.replace("\n", " ")
            # スクリーンネームを正規表現を用いて削除
            tweet1 = re.sub(r"@[0-9a-zA-Z_]{1,15} +", "", tweet1)

            tweet2 = id2status[rid].full_text.replace("\n", " ")
            tweet2 = re.sub(r"@[0-9a-zA-Z_]{1,15} +", "", tweet2)
            
            tag = age2tag(''.join(re.findall(pattern, user.description)[0]))

            f.write(tag + tweet1+ "\t" + tweet2 + "\n")
    f.close()
    print("Write " + str(len(id2replyid)) + " pairs.")


Write 576 pairs.


KeyboardInterrupt: ignored

# **分析**

上のコードで行っていること

1. ランダムにツイートを検索している（27～29行）
2. ツイートのスクリーンネームを取得して（29行）、そのユーザの行ったツイートを取得する。また、そのユーザのリプライ先のユーザのスクリーンネームを取得する（32～47行）。
3. リプライ先のスクリーンネームから、ユーザのツイートを取得する（50～56行）。
* 1,2,3を繰り返すことでツイートとリプライのペアを取得する。 

In [ ]:
api_key = 'yDw48aKG3Trf56AHoCNBp558l'
api_secret_key = '2XU9UfJgUGlhKJwDe7dIIHOJZ828EZqheWs8niCV6jvH5TxiZg'
access_token = '3041224710-AMaUeWnNloLRGcitnx5lAsIrdIh59jC5ehWkfeX'
access_token_secret = '4238EOYuICrESgWiH3ZDx35o8hiE1QWPFPx061hygy14u'

auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth_handler=auth, wait_on_rate_limit=True)

# botのツイートを除外するため，一般的なクライアント名を列挙
sources = ["TweetDeck", "Twitter Web Client", "Twitter for iPhone",
           "Twitter for iPad", "Twitter for Android", "Twitter for Android Tablets",
           "ついっぷる", "Janetter", "twicca", "Keitai Web", "Twitter for Mac"]
    

# ひらがな一文字で検索し，スクリーンネームを取得
words = list("あいうえおかきくけこさしすせそたちつてとなにぬねのはひふへほまみむめもやゆよらりるれろわをん")
    
# screen_name（ユーザー名）を取得している
screen_names = set()
for s in api.search(q=random.choice(words), lang='ja', result_type='recent', count=100, tweet_mode='extended'):
    if s.source in sources:
        screen_names.add(s.author.screen_name)

# ステータスidからステータスを得るためのdict
id2status = {}

# スクリーンネームからタイムラインを取得してツイートを保存．
# さらにリプライツイートであれば，リプライ先のスクリーンネームも取得
in_reply_to_screen_names = set()
for name in screen_names:
    try:
        for s in api.user_timeline(name, tweet_mode='extended', count=100):
            # リンクもしくはハッシュタグを含むツイートは除外する
            if "http" not in s.full_text and "#" not in s.full_text:
                id2status[s.id] = s
                if s.in_reply_to_screen_name is not None:
                    if s.in_reply_to_screen_name not in screen_names:
                        in_reply_to_screen_names.add(s.in_reply_to_screen_name)
    except Exception as e:
        continue

# リプライ先のスクリーンネームからタイムラインを取得してツイートを保存
for name in in_reply_to_screen_names:
    try:
        for s in api.user_timeline(name, tweet_mode='extended', count=100):
            if "http" not in s.full_text and "#" not in s.full_text:
                id2status[s.id] = s
    except Exception as e:
        continue

# 保存したツイートのリプライ先のツイートが保存されていれば，id2replyidのキーを元ツイートのid，値をリプライ先ツイートのidとする
id2replyid = {}
for _, s in id2status.items():
    if s.in_reply_to_status_id in id2status:
            id2replyid[s.in_reply_to_status_id] = s.id

# id2replyidのkey valueからstatusを取得し，ツイートペアをタブ区切りで保存
# 例）年齢を含むツイートを表示する
pattern = re.compile('[0-9]{2}[\u6B73\u4EE3]')
f = open("tweet_pairs.txt", "w")
for id, rid in id2replyid.items():
    user = api.get_user(screen_name=id2status[rid].author.screen_name)
    if re.search(pattern, user.description):
        # 改行は半角スペースに置換
        tweet1 = id2status[id].full_text.replace("\n", " ")
        # スクリーンネームを正規表現を用いて削除
        tweet1 = re.sub(r"@[0-9a-zA-Z_]{1,15} +", "", tweet1)

    
        tweet2 = id2status[rid].full_text.replace("\n", " ")
        tweet2 = re.sub(r"@[0-9a-zA-Z_]{1,15} +", "", tweet2)
        tweet2 = '<' + ''.join(re.findall(pattern, user.description)) + '>' + tweet2

        f.write(tweet1+ "\t" + tweet2 + "\n")
f.close()
print("Write " + str(len(id2replyid)) + " pairs.")

In [ ]:
pattern = re.compile('[0-9]+[\u6B73\u4EE3]')
for _, s in id2status.items():
    if s.in_reply_to_status_id in id2status:
        user = api.get_user(screen_name=s.author.screen_name)
        if re.search(pattern, user.description):
            tweet = s.full_text + '<' + ''.join(re.findall(pattern, user.description)) + '>'
            print(tweet)

In [ ]:
import re
pattern = re.compile('[1-6][0-9][\/\u6B73\u4EE3]')
s='10歳になりました30代の男です20/'
if re.search(pattern, s):
  print(re.findall(pattern, s))

['10歳', '30代', '20/']
